# Deepracer winner counter

<img src="https://github.com/aws-deepracer-community/deepracer-educational-resources/tree/main/rewards-notebook/img/0_dr_evo_360.gif" alt="EVO car" width="250">

## AWS DeepRacer
AWS DeepRacer is a 1/18th scale autonomous race car but also much more. It is a complete program that has helped thousands of employees in numerous organizations begin their educational journey into machine learning through fun and rivalry.

Visit [AWS DeepRacer page](http://deepracer.com/) to learn more about how it can help you and your organization begin and progress the journey towards machine learning.

Join the [AWS Machine Learning Community](http://join.deepracing.io/) to talk to people who have used DeepRacer in their learning experience.

## Prizes

AWS offer two types of prizes in the Pro Division of AWS:
1.  First Prize: AWS DeepRacer Championship qualification (24 total, 3 per month),
1.  Second Prize: AWS DeepRacer EVO (80 total, 10 per month),

But not everything straight forvard. Good racer could not get two cars or go win two slots in the Championships. There is "Prize condition": Each participant may receive a maximum of 1 of each prize type during the 2021 season.

Let's try and identify the winners using Python, Pandas and Jupyter Notebook.

## Data source

Fetching the data for this task could be a labour intensive task. There is no official DeepRacer API available.

Luckily Jochem Lugtenburg, a developer, AWS Community Builder and a DeepRacer expert in the AWS Machine Learning Community, has prepared a project gathering the data and exposing it to everyone in a GitHub repository: [deepracer-race-data](https://github.com/aws-deepracer-community/deepracer-race-data). We will use final race data for each month from this project.

## Data preparation

Let's import needed dependencies and load the data:

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

import urllib.request
from urllib.error import HTTPError

pd.set_option("display.max_rows", None, "display.max_columns", None)

There is a file [leaderboards.csv](https://github.com/aws-deepracer-community/deepracer-race-data/blob/main/raw_data/leaderboards/leaderboards.csv )  in the GitHub repository that contains information about all competitions: start date, end date, number of participants, rules, etc. We need to find and extract information about this season races. Let's take a closer look to this summary CSV file:

In [2]:
df_leaderboards = pd.read_csv('https://raw.githubusercontent.com/aws-deepracer-community/deepracer-race-data/main/raw_data/leaderboards/leaderboards.csv')
df_leaderboards.head()

,Arn,Name,Description,LaunchTime,CloseTime,StartTime,EndTime,TrackArn,Status,MinimumLaps,TotalLaps,ParticipantCount,ImageUrl,LeaderboardImage,RaceType,Visibility,TimingMethod,LeagueType,Moderated,TermsConditionId,WinnerAlias,HeadToHeadConfig_HeadToHeadType,HeadToHeadConfig_NumBotCars,HeadToHeadConfig_BotCarSpeed,HeadToHeadConfig_EnableLaneChange,HeadToHeadConfig_LowerLaneChangeTime,HeadToHeadConfig_UpperLaneChangeTime,ResettingBehaviorConfig_IsContinuousLap,ResettingBehaviorConfig_AllowReset,ResettingBehaviorConfig_MaxResetTimes,ResettingBehaviorConfig_PenaltySeconds,Tournament_Size,Tournament_ResultsReleaseTimes,PlayoffRaceTypeConfig_PlayoffRaceTypeID,ThemeConfig_ThemeID,ObjectAvoidanceConfig_RandomizeLocation,ObjectAvoidanceConfig_ObstaclePositions,ObjectAvoidanceConfig_NumObstacles,ObjectAvoidanceConfig_ObstacleType,ResettingBehaviorConfig_CollisionPenaltySeconds,ResettingBehaviorConfig_OffTrackPenaltySeconds,PlayoffRaceTypeConfig_PlayoffRaceArn,PlayoffRaceTypeConfig_PlayoffReleaseTime,PlayoffRaceTypeConfig_PlayoffResultReleaseTime,NumOfTeamSubmissionsRequired,ParticipationConfig_EntryTypeConfig_InvitationEntryType,ParticipationConfig_ViewType,CarControlConfig_CarSpeedControl,Tags,RacingDivision,ParticipationConfig_EntryTypeConfig_RacingDivisionEntryType
0,arn:aws:deepracer:us-east-1::leaderboard/prese...,Summit Speedway Pre-season,"Take part in the Summit Speedway, the pre-seas...",1555516800000,1556550000000,1555516800000,1556550000000,arn:aws:deepracer:us-east-1::track/Virtual_Com...,CLOSED,3.0,5.0,162,https://deepracer-managed-resources-us-east-1....,https://deepracer-managed-resources-us-east-1....,TIME_TRIAL,VISIBLE,AVG_LAP_TIME,VIRTUAL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,arn:aws:deepracer:us-east-1::leaderboard/seaso...,London Loop,"Race in the London Loop, the first stop of the...",1556553600000,1559347199000,1556553600000,1559347199000,arn:aws:deepracer:us-east-1::track/Virtual_May...,CLOSED,2.0,5.0,859,https://deepracer-managed-resources-us-east-1....,https://deepracer-managed-resources-us-east-1....,TIME_TRIAL,VISIBLE,AVG_LAP_TIME,VIRTUAL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,arn:aws:deepracer:us-east-1::leaderboard/seaso...,Kumo Torakku,Continue racing and building up League points ...,1559595600000,1561964399000,1559595600000,1561964399000,arn:aws:deepracer:us-east-1::track/Tokyo_Racin...,CLOSED,1.0,5.0,572,https://deepracer-managed-resources-us-east-1....,https://deepracer-managed-resources-us-east-1....,TIME_TRIAL,VISIBLE,AVG_LAP_TIME,VIRTUAL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,arn:aws:deepracer:us-east-1::leaderboard/seaso...,Empire City Circuit,The July race is now open and is the third sto...,1561964400000,1564599599000,1561964400000,1564599599000,arn:aws:deepracer:us-east-1::track/New_YorkAlt...,CLOSED,1.0,5.0,650,https://deepracer-managed-resources-us-east-1....,https://deepracer-managed-resources-us-east-1....,TIME_TRIAL,VISIBLE,AVG_LAP_TIME,VIRTUAL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,arn:aws:deepracer:us-east-1::leaderboard/seaso...,Shanghai Sudu,The August race is now open and is the fourth ...,1564642800000,1567295999000,1564642800000,1567295999000,arn:aws:deepracer:us-east-1::track/ChinaAlt_track,CLOSED,1.0,5.0,1375,https://deepracer-managed-resources-us-east-1....,https://deepracer-managed-resources-us-east-1....,TIME_TRIAL,VISIBLE,AVG_LAP_TIME,VIRTUAL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


You need to manually add data to variable **month_races** which is colomn "Name" from df_leaderboards: [leaderboards.csv](https://github.com/aws-deepracer-community/deepracer-race-data/blob/main/raw_data/leaderboards/leaderboards.csv ) 

In [3]:
months_races = ['March Qualifier', 'April Qualifier', 'May Qualifier', 'June Qualifier', 'July Qualifier', 'August Qualifier', 'September Qualifier', 'October Qualifier']

race_type = ['HEAD_TO_HEAD_RACING', 'TIME_TRIAL', 'OBJECT_AVOIDANCE']

# Get Arn URLs according to race type: HEAD_TO_HEAD_RACING – month qualifier leader board and OBJECT_AVOIDANCE – final race
month_leaderboard_arn = df_leaderboards.loc[(df_leaderboards['Name'].isin(months_races)) & (df_leaderboards['RaceType'] == "HEAD_TO_HEAD_RACING")]['Arn'].values
month_final_arn = df_leaderboards.loc[(df_leaderboards['Name'].isin(months_races)) & (df_leaderboards['RaceType'] == "OBJECT_AVOIDANCE")]['Arn'].values

# Next we need to get raw data for final races and month qualifier leader. We put dataframes to lists: one for month qualifier and second for winners
path = "https://raw.githubusercontent.com/aws-deepracer-community/deepracer-race-data/main/raw_data/leaderboards/"
suffix = "/FINAL.csv"

list_end_of_month=[]
for arn_leaderboard in month_leaderboard_arn:
  try: 
    list_end_of_month.append(pd.read_csv(path+month_leaderboard_arn[np.where(month_leaderboard_arn == arn_leaderboard)[0][0]].replace(":", "%3A")+suffix))
  except urllib.error.HTTPError as err:
    list_end_of_month.append(pd.DataFrame(columns=['Alias', 'UserId', 'Rank', 'Month']))


list_finale=[]
for arn_win in month_final_arn:
  try:
    list_finale.append(pd.read_csv(path+month_final_arn[np.where(month_final_arn == arn_win)[0][0]].replace(":", "%3A")+suffix))
  except urllib.error.HTTPError as err:
    list_finale.append(pd.DataFrame(columns=['Alias', 'UserId', 'Rank', 'Month']))
    pass

You can mention construction 
```
try:
except urllib.error.HTTPError as err:
```
that was used due to Final october race still upcoming at the time of writing this article. Because of that the file FINAL.csv was not available and we protected ourselves by adding an empty dataframe so that we didn't have to handle a missing entry in a list in code that followed


Let's have a quick look at what information we can get from the repository:

In [4]:
# Index 0 is foe March, respectively 1 is for April etc.
list_finale[0].head()

,Alias,SubmissionTime,LapCount,AvgLapTime,BestLapTime,TotalLapTime,AvgResets,CollisionCount,OffTrackCount,ResetCount,SubmissionCount,Rank,LeaderboardSubmissionStatusType,RankingScore,RacingDivision,UserId
0,Karl-NAB,1617932584399,1,31151.0,31151.0,164206.0,0.20,0,1,1,0,1,SUCCESS,31151,OPEN,4vOOxLIsSjeL2WFaO5o_fQ
1,JJ,1617932801469,1,32864.0,32864.0,149725.0,0.75,0,3,3,0,2,SUCCESS,32864,OPEN,afdcX_4vRH6Ussd0mbg5LQ
2,DBro,1617932943399,1,33338.0,33338.0,171756.0,1.25,3,2,5,4,3,SUCCESS,33338,OPEN,jEkEOpjZRhGUbyDZIfTo_A
3,flatearth,1617918588918,1,34899.0,34899.0,156114.0,3.00,7,2,9,0,4,SUCCESS,34899,OPEN,9e3a9607a4e04ba2b1a49w
4,PolishThunder,1617932614670,1,36618.0,36618.0,141956.0,2.00,1,5,6,0,5,SUCCESS,36618,OPEN,qi512oGVTry8V-KtP6j2_w


In reality we only care about three pieces of data: Alias, Rank and and UserId.

Let's have a look at a narrowed dataset:

In [5]:
list_finale[0][['Alias', 'UserId', 'Rank']].head()

,Alias,UserId,Rank
0,Karl-NAB,4vOOxLIsSjeL2WFaO5o_fQ,1
1,JJ,afdcX_4vRH6Ussd0mbg5LQ,2
2,DBro,jEkEOpjZRhGUbyDZIfTo_A,3
3,flatearth,9e3a9607a4e04ba2b1a49w,4
4,PolishThunder,qi512oGVTry8V-KtP6j2_w,5


## Finalists

Let's start with the finalists. After each month's Pro Division race, top 16 racers who have not yet qualified into the championships compete in a finale race. Top 3 racers from each such race qualify into the championships.

`df_month_finale` dataframes above hold results for those races. They are sorted by Rank which makes it easier for us as we don't have to think about reordering the records.

To build a list of winners, for each month we need to take the top 3 racers and append them to a list of racers. Luckily we don't neet to worry about duplicate racers either as none of the previous winners take part in the finales anymore.

![DeepRacer.jpeg](./img/1_winners_selection.jpeg)

This is our method to determine the finalists:







In [6]:
Month = ['March', 'April', 'May', 'June', 'July', 'August', 'September', 'October']

def championship_racers():
  df_winners = pd.DataFrame(columns=['Alias', 'UserId', 'Rank', 'Month'])

  for idx, finale in enumerate(list_finale):
    df_winners = df_winners.append(finale[['Alias', 'UserId', 'Rank']].iloc[:3]).reset_index(drop=True)
    df_winners['Month']= df_winners['Month'].fillna(Month[idx])
  return df_winners

A few words on what we did here:
* We've prepared an empty Pandas dataframe with columns
* We enumerated over the list of dataframes - `enumerate` method iterates over the elements of a list but it also provides an index of the element in that list. This helps us add the month name as an extra column
* We've added a month column, just to know in which month this person qualified
* We've simply appended top three rows from each finale dataframe into the winners dataframe. We used `iloc` to limit the number of rows
* We've reset the index values. Without this the new dataframe with winners would have the index all messed up
* To add the months we use a `fillna` method. It's pretty handy as it only sets values where they are missing

Let's see the results:

In [7]:
championship_racers()

,Alias,UserId,Rank,Month
0,Karl-NAB,4vOOxLIsSjeL2WFaO5o_fQ,1,March
1,JJ,afdcX_4vRH6Ussd0mbg5LQ,2,March
2,DBro,jEkEOpjZRhGUbyDZIfTo_A,3,March
3,JPMC-RogerRabbit,DXXOncNRR1iiqhBvpPO-_A,1,April
4,flatearth,9e3a9607a4e04ba2b1a49w,2,April
5,PolishThunder,qi512oGVTry8V-KtP6j2_w,3,April
6,Jochem,RBPhZc5eTVmEF_gpSV7skg,1,May
7,JPMC-DriftKing-Houston,fngewpiZSZm11yX7YgwnMA,2,May
8,Joffe,ce17e3bfe82649448e2f3w,3,May
9,JPMC-TeamOne-BuenosAires,Fy8bvUIzSN-N-e3D74JhPw,1,June


Let's imagine that the month just finished and the final race have not occure yet but we want to see who will compete in it. Let's see who compete in October final race:

In [8]:
list_end_of_month[7][['Alias', 'UserId', 'Rank']].head(n=16)

,Alias,UserId,Rank
0,JJ,afdcX_4vRH6Ussd0mbg5LQ,1
1,Karl-NAB,4vOOxLIsSjeL2WFaO5o_fQ,2
2,rosscomp1,fb0d597a87f94697aa1c6w,3
3,JPMC-Rogue-Hyderabad,LKdw_hyKTSu7moFuxbmiZw,4
4,Duckworth,VIu9BQ4-Th6gUqu60ztwUQ,5
5,Ghost-Rider-007,yjfHxWGORWWg1wHFQFBH4g,6
6,PolishThunder,qi512oGVTry8V-KtP6j2_w,7
7,Ernesto,8zBEAp0-S5yGbcZvytGZGQ,8
8,Yi-LI-NYCU-CGI,jrJHq8pZQAyj4gGqMw8-DQ,9
9,Fumiaki,5RzBNYRAQoWVausi3eyhRg,10


Well, this is wrong - I can see people who have already qualified into the championships on this list.

To determine the finale racers we need to use October race results, but we first need to remove those who already qualified. Let's try that:

In [9]:
def october_finale_racers():
  championship_racers_so_far = championship_racers()[['Alias', 'UserId', 'Rank']]
  return list_end_of_month[7][['Alias', 'UserId', 'Rank']].append(championship_racers_so_far).drop_duplicates('UserId', keep='last').reset_index(drop=True).head(n=16)

We've use a new method: `drop_duplicates`. It takes values of specified column and by default keeps the first one and discards the rest. We used a little trick here - by putting the championship racers at the end and telling the `drop_duplicates` to only keep the last appearance we make sure that if the they raced in October, their entry will be dropped, leaving only those who can join the top finale race.

UserId is a unique and unchangeable identifier for each racer. Starting this season users can change their Aliases (which comes as a great relief to those racing under their often very creative model names) so trying to enforce uniqueness using the Alias values would not work.

Let's run this:

In [10]:
october_finale_racers()

,Alias,UserId,Rank
0,Ghost-Rider-007,yjfHxWGORWWg1wHFQFBH4g,6
1,Ernesto,8zBEAp0-S5yGbcZvytGZGQ,8
2,Fumiaki,5RzBNYRAQoWVausi3eyhRg,10
3,SorinB,TFtvCas4TRWchrBPMyGh0A,13
4,GT-DevelopersIO,8WSp4P7ZRn-jJgZLnugwkA,14
5,AJM-Model-1,gdsU9bkGRWW7LTEHP4rGgQ,16
6,JPMC-HYD-CircuitBreakers,vA_gK0vxTNWI-MTWEWLAxw,17
7,TonyJ,af962be5a4ed4c57af807w,18
8,CGS-BigBoson,U1DMLV72SSmIjLE-fPGbuw,20
9,Deepak-dpk,gDTBFK77TM6wD6uObZlulA,21


## AWS DeepRacer Evo winners

Now time for the Evo winners. Here rules get a little bit more complicated. As written above, it is top 10 racers that win the car and only once. In March we care about top 10 finale racers. In April - some from finale, but maybe we have someone from below the top 16 that wins one?

We need to take the month's race results into consideration but also the fact that someone from places 10-16 in that race might be in top 10 in the finale.

Let's have a look at the code and then we'll look at what's going on in here:

In [11]:
def car_winners():
  df_car_winners = pd.DataFrame(columns=['Alias', 'UserId', 'Rank', 'Month'])
  for idx, list_file in enumerate(list_end_of_month):
    df_car_winners = df_car_winners.append(list_finale[idx][['Alias', 'UserId', 'Rank']]).append(list_file[['Alias', 'UserId', 'Rank']]).drop_duplicates('UserId').reset_index(drop=True).iloc[:(idx+1)*10]
    df_car_winners['Month']= df_car_winners['Month'].fillna(Month[idx])
  return df_car_winners


Let's focus at this bit as it's the most important here:

```
df_car_winners.append(list_finale[idx][['Alias', 'UserId', 'Rank']]).append(list_file[['Alias', 'UserId', 'Rank']]).drop_duplicates('UserId')
```

For each month we take the car winners so far, append the finale results to it, then the month race results, and drop the duplicates leaving only the first occurrence of a UserId value. This means that if someone is already a car winner, their entries will be removed from finale and month race results. Likewise, if they were finale racers, their month race results will be removed.

Effectively this means tha we build a list containing all the car winners so far and a list of rank-ordered performers in a given month who have not yet won a car. All we need to do now is to drop everyone except of the top ten for a given month. Since each month the list is growing, we perform `.iloc[:(idx+1)*10]`.

Let's see the results:

In [12]:
car_winners()

,Alias,UserId,Rank,Month
0,Karl-NAB,4vOOxLIsSjeL2WFaO5o_fQ,1,March
1,JJ,afdcX_4vRH6Ussd0mbg5LQ,2,March
2,DBro,jEkEOpjZRhGUbyDZIfTo_A,3,March
3,flatearth,9e3a9607a4e04ba2b1a49w,4,March
4,PolishThunder,qi512oGVTry8V-KtP6j2_w,5,March
5,Jochem,RBPhZc5eTVmEF_gpSV7skg,6,March
6,Maikel,FeKoIRn0Tsa6tACAWZ_2gw,7,March
7,GT-DevelopersIO,8WSp4P7ZRn-jJgZLnugwkA,8,March
8,Ernesto,8zBEAp0-S5yGbcZvytGZGQ,9,March
9,JPMC-RogerRabbit,DXXOncNRR1iiqhBvpPO-_A,10,March


## Wildcard race winners

AWS always give a last minute opportunity to qualify. Normally this would be a live race at the re:Invent conference but since everything is taking place virtually, so is this Wildcard race. Top five participants take part in the championships, but we need to sift out those who race but already had their places secured.

The race just finished and we wanted to know who qualified. We can either add FINAL.csv in the months_races list or **get raw** information from the Github table. Click "raw" and copy URL: 
<img src="img/2_raw_file_finding_on_github.png" />

Now all that's left is to load the file, remove the finalists so far and list top five racers:

In [13]:
wildcard_open = pd.read_csv("https://raw.githubusercontent.com/aws-deepracer-community/deepracer-race-data/main/raw_data/leaderboards/arn%3Aaws%3Adeepracer%3A%3A%3Aleaderboard/08db3006-f491-48b4-a238-926c6465e5d8/FINAL.csv")

def wildcard_qualifier(df_wildcard):
  winners = championship_racers()
  wildcard_5 = df_wildcard[['Alias', 'UserId', 'Rank']].append(winners).drop_duplicates('UserId', keep='last').reset_index(drop=True).head(n=5)
  wildcard_5['Month']= wildcard_5['Month'].fillna("wildcard")   
  return wildcard_5

Let's see who quilified through the Wildcard race:

In [14]:
wildcard_qualifier(wildcard_open)

,Alias,UserId,Rank,Month
0,AJM-Model-1,gdsU9bkGRWW7LTEHP4rGgQ,3,wildcard
1,JPMC-Ace-Hyderabad,6fdcVCAkQtCCsyHEO0LNiA,4,wildcard
2,DoomBuggy,QakV_nzOTm2q_NhqUMwIJw,6,wildcard
3,JPMC-MUMBAI-FormulaAI,Gx2StgEoSza3Gxxo4Oghug,8,wildcard
4,Ernesto,8zBEAp0-S5yGbcZvytGZGQ,9,wildcard


## Limitations

There are things we cannot verify ourselves that may greatly influence the above results. AWS perform eligibility checks on those racers and the racers themselves need to claim the prize. This means that racers can get removed from this list and we have no way do find out.

## Source code

This article has been prepared as a Jupyter Notebook which we have shared on GitHub. You can download it yourself and play with it a little here: [https://github.com/mokoron/deepracer.git](https://github.com/mokoron/deepracer.git)

# SageMaker 

I guess you want to play aroud with notebook. You can easily do it by running this into AWS Sagemaker. 

First, login into your ASW account and navigate to Amazon SageMaker:

![How to find SageMaker in AWS Console](./img/3_sagemaker_in_aws_console.png)

Navigate to Notebook – Notebook Instancies and Create notebook instance (orange button on a right top corner). Choose a name for Notebook instance other parameters you can leave as default value.

![Creating a new SageMaker Notebook](./img/4_new_sagemaker_notebook.png)

Next, in the Git Repositories section select "Clone a public Git repository to this instance only" and provide the URL of this article's repository: 
![Selecting a public repository](./img/5_clone_git_repository.png)

Wait a couple of minutes for Instance to be created.

Now you can open the notebook and make changes to it. If you get an error about kernel not being found, select conda_python3

Do not forget to Stop Notebook instance when you finish to prevent unexpected billing. **YOU WILL GET BILLED IF YOU DO NOT**

![Stopping a SageMaker Notebook to prevent costs](./img/6_stop_sagemaker_notebook.png)